In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import os
%matplotlib inline
import tensorflow as tf
import datetime
from keras.callbacks import ModelCheckpoint, CSVLogger, TensorBoard, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img,img_to_array


In [19]:
tf.test.is_gpu_available

<function tensorflow.python.framework.test_util.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)>

In [2]:
train_path_gen=r"C:\Users\lenovo\Desktop\male-female-faceDataset-main\Training"
val_path_gen=r"C:\Users\lenovo\Desktop\male-female-faceDataset-main\Validation"

In [3]:
epochs = 50
lr = 1e-3
batchSize=128
data = []
labels = []
size= 224 #height/width

In [4]:
trainDatagen = ImageDataGenerator(horizontal_flip=True,
                                  width_shift_range=0.4,
                                  height_shift_range=0.4,
                                  zoom_range=0.3,
                                  rotation_range=20,
                                  rescale=1/255
                                  )

In [5]:
testGen = ImageDataGenerator(rescale=1/255)
targetSize=(size,size)

In [6]:
trainGen = trainDatagen.flow_from_directory(
    directory=train_path_gen,
    target_size=targetSize,
    batch_size=batchSize,
    class_mode ='binary'
)

Found 23829 images belonging to 2 classes.


In [7]:
validationGen= testGen.flow_from_directory(
    directory=val_path_gen,
    target_size=targetSize,
    batch_size=batchSize,
    class_mode ='binary'
    )

Found 7199 images belonging to 2 classes.


In [8]:
model=Sequential()
model.add(InceptionV3(include_top=False,pooling='avg',weights='imagenet'))
model.add(Flatten())

model.add(BatchNormalization())
model.add(Dense(2048,activation='relu'))
model.add(BatchNormalization())

model.add(Dense(1024,activation='relu'))
model.add(BatchNormalization())

model.add(Dense(1,activation='sigmoid'))

model.layers[0].trainable=False

In [9]:
model.compile(optimizer='adam',
              loss = 'binary_crossentropy',
              metrics=['accuracy'])

In [10]:
chk_path = 'GenNet.h5'
log_dir = "checkpoint/logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

checkpoint = ModelCheckpoint(filepath=chk_path,
                             save_best_only=True,
                             verbose=1,
                             mode='min',
                             moniter='val_loss')

earlystop = EarlyStopping(monitor='val_loss', 
                          min_delta=0, 
                          patience=3, 
                          verbose=1, 
                          restore_best_weights=True)
                        
reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
                              factor=0.2, 
                              patience=6, 
                              verbose=1, 
                              min_delta=0.0001)


tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
csv_logger = CSVLogger('training.log')

callbacks = [checkpoint, reduce_lr, csv_logger]

In [12]:
hist=model.fit(trainGen,steps_per_epoch=len(trainGen.filenames)//batchSize,
          epochs=5,
          validation_data=validationGen,
          validation_steps=len(validationGen.filenames)//batchSize,
          callbacks=callbacks)

Epoch 1/5
  1/186 [..............................] - ETA: 50:15 - loss: 1.0926 - accuracy: 0.7656

KeyboardInterrupt: 